<a href="https://colab.research.google.com/github/liupengzhouyi/LearningPytorch/blob/master/20200325/ResNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ResNet

## import package

In [0]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

## setting Device

In [0]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## settting hyper parameters

In [0]:
# Hyper-parameters
num_epochs = 5
batch_size = 100
learning_rate = 0.001

## setting download

In [0]:
# Image preprocessing modules
transform = transforms.Compose([
    transforms.Pad(4),
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32),
    transforms.ToTensor()])

## Download Train Data

In [5]:
# CIFAR-10 dataset
train_dataset = torchvision.datasets.CIFAR10(root='../../data/',
                                             train=True, 
                                             transform=transform,
                                             download=True)

Extracting ../../data/cifar-10-python.tar.gz to ../../data/


## Train data To Train Data Set


In [0]:
# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

## Download Test Data

In [0]:
# CIFAR-10 dataset
test_dataset = torchvision.datasets.CIFAR10(root='../../data/',
                                            train=False, 
                                            transform=transforms.ToTensor())

## Test Data To Test Data Set

In [0]:
# Data loader
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

## Create Convolution Which Size Was 3*3

In [0]:
# 3x3 convolution
def conv3x3(in_channels, out_channels, stride=1):
    return nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)

## Create Residual Block

In [0]:
# Residual block
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super(ResidualBlock, self).__init__()
        self.conv1 = conv3x3(in_channels, out_channels, stride)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(out_channels, out_channels)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.downsample = downsample
        
    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        if self.downsample:
            residual = self.downsample(x)
        out += residual
        out = self.relu(out)
        return out

## Create ResNet

In [0]:
# ResNet
class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes=10):
        super(ResNet, self).__init__()
        self.in_channels = 16
        self.conv = conv3x3(3, 16)
        self.bn = nn.BatchNorm2d(16)
        self.relu = nn.ReLU(inplace=True)
        self.layer1 = self.make_layer(block, 16, layers[0])
        self.layer2 = self.make_layer(block, 32, layers[1], 2)
        self.layer3 = self.make_layer(block, 64, layers[2], 2)
        self.avg_pool = nn.AvgPool2d(8)
        self.fc = nn.Linear(64, num_classes)
        
    def make_layer(self, block, out_channels, blocks, stride=1):
        downsample = None
        if (stride != 1) or (self.in_channels != out_channels):
            downsample = nn.Sequential(
                conv3x3(self.in_channels, out_channels, stride=stride),
                nn.BatchNorm2d(out_channels),
            )
        layers = []
        layers.append(block(self.in_channels, out_channels, stride, downsample))
        self.in_channels = out_channels
        for i in range(1, blocks):
            layers.append(block(out_channels, out_channels))
        return nn.Sequential(*layers)
    
    def forward(self, x):
        out = self.conv(x)
        out = self.bn(out)
        out = self.relu(out)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.avg_pool(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

## Create ResNet Model

In [0]:
model = ResNet(ResidualBlock, [2, 2, 2]).to(device)

## Create Loss Function

In [0]:
criterion = nn.CrossEntropyLoss()

## Creaet Optimizer

In [0]:
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

## A Function Of Update Our learning rate

In [0]:
# For updating learning rate
def update_lr(optimizer, lr):    
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

## Train Model

In [16]:
# Train the model
total_step = len(train_loader)
curr_lr = learning_rate
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ("Epoch [{}/{}], Step [{}/{}] Loss: {:.4f}"
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

    # Decay learning rate
    if (epoch+1) % 20 == 0:
        curr_lr /= 3
        update_lr(optimizer, curr_lr)

Epoch [1/5], Step [100/500] Loss: 1.6791
Epoch [1/5], Step [200/500] Loss: 1.3624
Epoch [1/5], Step [300/500] Loss: 1.1635
Epoch [1/5], Step [400/500] Loss: 1.1799
Epoch [1/5], Step [500/500] Loss: 1.0369
Epoch [2/5], Step [100/500] Loss: 1.0040
Epoch [2/5], Step [200/500] Loss: 1.2379
Epoch [2/5], Step [300/500] Loss: 1.0537
Epoch [2/5], Step [400/500] Loss: 1.1993
Epoch [2/5], Step [500/500] Loss: 0.9649
Epoch [3/5], Step [100/500] Loss: 1.0037
Epoch [3/5], Step [200/500] Loss: 0.7084
Epoch [3/5], Step [300/500] Loss: 0.7244
Epoch [3/5], Step [400/500] Loss: 0.8252
Epoch [3/5], Step [500/500] Loss: 0.9207
Epoch [4/5], Step [100/500] Loss: 0.8508
Epoch [4/5], Step [200/500] Loss: 0.9260
Epoch [4/5], Step [300/500] Loss: 0.7778
Epoch [4/5], Step [400/500] Loss: 0.7100
Epoch [4/5], Step [500/500] Loss: 0.8680
Epoch [5/5], Step [100/500] Loss: 0.6112
Epoch [5/5], Step [200/500] Loss: 0.4943
Epoch [5/5], Step [300/500] Loss: 0.7333
Epoch [5/5], Step [400/500] Loss: 0.4697
Epoch [5/5], Ste

## Test Model

In [17]:
# Test the model
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy of the model on the test images: {} %'.format(100 * correct / total))

Accuracy of the model on the test images: 74.59 %


## Save Model

In [0]:
# Save the model checkpoint
torch.save(model.state_dict(), 'resnet.ckpt')